# 환경설정

In [1]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
from bs4 import BeautifulSoup
import re
import json
#options = webdriver.ChromeOptions()
#options.add_argument('headless')

# 크롬창 열기

In [2]:
url = 'https://map.naver.com/v5/search'
driver = webdriver.Chrome('./chromedriver')  # 드라이버 경로
#driver = webdriver.Chrome('./chromedriver',chrome_options=options) # 크롬창 숨기기
driver.get(url)
key_word_list = ['노량진수산시장 횟집', '송파구 횟집', '강서구 횟집', '강남구 횟집', '노원구 횟집', '은평구 횟집']

C:\Users\JSWonner\AppData\Local\Temp\ipykernel_1724\2096135440.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')  # 드라이버 경로


In [3]:
key_word = ['노량진수산시장 횟집']

In [4]:
# css 찾을때 까지 10초대기
def time_wait(num, code):
    try:
        wait = WebDriverWait(driver, num).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, code)))
    except:
        print(code, '태그를 찾지 못하였습니다.')
        driver.quit()
    return wait

In [5]:
def find_search_input(key_word):
    # 검색창 찾기
    time_wait(2, 'div.input_box > input.input_search')
    search = driver.find_element(By.CSS_SELECTOR,'div.input_box > input.input_search')
    search.send_keys(key_word)  # 검색어 입력
    search.send_keys(Keys.ENTER)  # 엔터버튼 누르기
    
    res = driver.page_source  # 페이지 소스 가져오기
    soup = BeautifulSoup(res, 'html.parser')  # html 파싱하여  가져온다
    sleep(1)

In [6]:
# frame 변경 메소드
def switch_frame(frame):
    driver.switch_to.default_content()  # frame 초기화
    driver.switch_to.frame(frame)  # frame 변경
    driver.implicitly_wait(1)


In [7]:
# 페이지 다운
def page_down(num):
    body = driver.find_element(By.CSS_SELECTOR,'body')
    body.click()
    for i in range(num):
        body.send_keys(Keys.PAGE_DOWN)

In [8]:
def find_store_list():
    # 매장 리스트
    store_list = driver.find_elements(By.CSS_SELECTOR,'._22p-O')
    # 페이지 리스트
    next_btn = driver.find_elements(By.CSS_SELECTOR,'._2ky45 > a')
    return store_list, next_btn

In [10]:
def find_review_btn():
    sleep(2)
    time_wait(10,'a._2RG_o._tab-menu')
    menu_list = driver.find_elements(By.CSS_SELECTOR,'a._2RG_o._tab-menu')
    if len(menu_list) ==7:
        return menu_list[-1]
    else:
        return menu_list[-2]

In [ ]:
def crawling_visitor_review():
    #-----리뷰 버튼 클릭-------
    find_review_btn().click()
    SCROLL_PAUSE_SEC = 1
    
    last_scroll = driver.execute_script("return document.body.scrollHeight")
    
    
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_SEC)
        
        try :
            #더보기 버튼 클릭
            driver.find_element(By.CSS_SELECTOR,'._3iTUo').click()
            
        except:
            #더보기 버튼이 없다면 while문을 빠져나온다.
            break
            # 끝까지 스크롤 다운
            
        new_scroll = driver.execute_script("return document.body.scrollHeight")
        if last_scroll == new_scroll:
            break
        else:
            last_scroll= new_scroll



    #가게 리스트 마다 방문자 리뷰 가져오기
    review_elements = driver.find_elements(By.CSS_SELECTOR,"div.faZHB > a > span")
    review_list = []
    for i in review_elements:
        review_list.append(i.text)
    
    return review_list


In [93]:
def crawling_blog_review():
    driver.execute_script("window.scrollTo(0, 0);")
    sleep(2)
    try :
        blog_review_button = driver.find_element(By.CSS_SELECTOR, 'div._1tclc > div > a:nth-child(2)')
        blog_review_button.click()
    except:
        print('블로그 리뷰가 없습니다.')
        return
    
    SCROLL_PAUSE_SEC = 1
    sleep(SCROLL_PAUSE_SEC)
    
    last_scroll = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_SEC)
        
        try :
            #더보기 버튼 클릭
            driver.find_element(By.CSS_SELECTOR,'._3iTUo').click()
        except:
            break
            # 끝까지 스크롤 다운
            
        new_scroll = driver.execute_script("return document.body.scrollHeight")
        if last_scroll == new_scroll:
            break
        else:
            last_scroll = new_scroll
    
    
    #가게 리스트 마다 방문자 리뷰 가져오기
    review_elements = driver.find_elements(By.CSS_SELECTOR,'div._3jvFk')
    review_list = []
    for i in review_elements:
        review_list.append(i.text)
    return review_list


# 검색 지역 횟집의 리뷰를 크롤링.

In [94]:
review_dic = {}
def review_crawling(key_word):
    #검색창을 찾아서 서치
    find_search_input(key_word)
    # searchIframe 변경
    switch_frame('searchIframe')
    driver.implicitly_wait(2)
    # 페이지 버튼의 수
    pages = driver.find_elements(By.CSS_SELECTOR,'a._2tk2s') # 이건 필요
    for btn in range(1,4): #페이지 버튼
         #페이지 버튼 클릭
        #각 매장 요소들의 하이퍼링크 리스트
        page_down(40)
        driver.implicitly_wait(2)
        store_elements = driver.find_elements(By.CSS_SELECTOR,'div._3ZU00._1rBq3 > a:nth-child(1)')
        
        #한 페이지의 50개의 매장들 하나씩 조회
        for data in range(len(store_elements)): 
            store_elements[data].click() #li의 페이지를 클릭
            driver.implicitly_wait(2) # 2초 휴식
            try:
                #다른 iframe으로 변경
                switch_frame('entryIframe')
                time_wait(2, '._3XamX')
        
                # -----매장명 가져오기-----
                store_name = driver.find_element(By.CSS_SELECTOR,'._3XamX').text
                driver.implicitly_wait(2)
                # -----리뷰 크롤링---------
                review_dic[store_name] = crawling_visitor_review()
                sleep(5)
                review_dic[store_name] = review_dic[store_name]+crawling_blog_review()
                sleep(5)
                
            except:
                pass
            switch_frame('searchIframe')
            
        pages[btn].click()
            
        # json 파일로 저장
    with open('review_dic.json', 'w', encoding='utf-8') as f:
        json.dump(review_dic, f, indent=4, ensure_ascii=False)
        
            

In [95]:
url = 'https://map.naver.com/v5/search'
driver = webdriver.Chrome('./chromedriver')  # 드라이버 경로
#driver = webdriver.Chrome('./chromedriver',chrome_options=options) # 크롬창 숨기기
driver.get(url)
key_word = '노량진수상시장 횟집'

C:\Users\JSWonner\AppData\Local\Temp\ipykernel_13360\3270057109.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')  # 드라이버 경로


In [ ]:
review_crawling(key_word)

In [74]:
url = 'https://map.naver.com/v5/search'
driver = webdriver.Chrome('./chromedriver')  # 드라이버 경로

C:\Users\JSWonner\AppData\Local\Temp\ipykernel_13360\505887123.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')  # 드라이버 경로


In [75]:
driver.get(url)
key_word = '노량진수상시장 횟집'
find_search_input(key_word)

In [76]:
switch_frame('searchIframe')

In [77]:
driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")

In [78]:
page_down(3)

In [79]:
switch_frame('entryIframe')

In [80]:
driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")

In [81]:
driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")

In [82]:
switch_frame('searchIframe')

In [83]:
driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")

In [84]:
page_down(20)

In [85]:
switch_frame('entryIframe')

In [36]:
driver.execute_script("window.scrollTo(0,0);")

In [86]:
driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")

In [88]:
driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")